In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2  
from tqdm import tqdm
from random import randint
import json
import matplotlib.image as mpimg
from matplotlib import rcParams
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from tensorflow.keras.applications.vgg16 import preprocess_input,VGG16

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Using TensorFlow backend.


In [2]:
train_path = "data/train"
test_path = "data/test"

In [3]:
train = os.listdir(train_path)
test = os.listdir(test_path)

In [4]:
train_image = [cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(train_path, image)), cv2.COLOR_BGR2RGB),(150,150))  for image in train ]

In [5]:
test_image = [cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(test_path, image)), cv2.COLOR_BGR2RGB),(150,150))  for image in test ]

In [6]:
label = pd.read_csv("data/train_features.csv")
label[["x","y"]]

,x,y
0,-9.380678,3.58272
1,-9.380678,3.58272
2,-9.380678,3.58272
3,-9.380678,3.58272
4,-9.380678,3.58272
...,...,...
7495,-4.680678,35.18272
7496,-4.680678,35.18272
7497,-4.680678,35.18272
7498,-4.680678,35.18272


In [7]:
y_label = []
for index, row in label.iterrows():
    y_label.append((row["x"],row["y"]))
y_label = np.array(y_label)
train_image = np.array(train_image)

In [8]:
test_image = np.array(test_image)

In [9]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [10]:
#vgg16 = VGG16(weights='imagenet',input_shape=[150,150,3],include_top=False)
#vgg19 = VGG19(weights='imagenet',input_shape=[150,150,3],include_top=False)
resnet50 = ResNet50(weights='imagenet',input_shape=[150,150,3],include_top=False)

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers.merge import concatenate
from tensorflow.keras.models import Model

In [16]:
def train_model_full(pre_train):
  for layer in pre_train.layers:
    layer.trainable = False
  layer = layers.Conv2D(filters = 32,kernel_size = (1,1),padding = "same",activation = "relu")(pre_train.layers[-1].output)
  layer = layers.MaxPooling2D(pool_size=(2,2), strides=2,padding="same")(layer)
  layer = layers.Conv2D(filters = 64,kernel_size = (3,3),padding = "same",activation = "relu")(layer)
  layer = layers.Flatten()(layer)
  layer = layers.Dense(1024,activation="relu")(layer)
  layer = layers.Dense(2,activation="linear")(layer)

  model = tf.keras.Model(inputs = pre_train.inputs,outputs = layer)

  return model

In [17]:
opt = keras.optimizers.Adam(learning_rate=0.001)


In [18]:
model_resnet50_full = train_model_full(resnet50)

model_resnet50_full.compile(optimizer=opt,
            loss=tf.keras.losses.MeanAbsoluteError())

In [19]:
model_resnet50_full.fit(train_image, label[["x","y"]], epochs=40, batch_size=128)

Epoch 1/40
59/59 [==============================] - 15s 217ms/step - loss: 32.0646
Epoch 2/40
59/59 [==============================] - 13s 216ms/step - loss: 22.9276
Epoch 3/40
59/59 [==============================] - 11s 185ms/step - loss: 19.0102
Epoch 4/40
59/59 [==============================] - 11s 182ms/step - loss: 16.3629
Epoch 5/40
59/59 [==============================] - 11s 184ms/step - loss: 14.5173
Epoch 6/40
59/59 [==============================] - 11s 193ms/step - loss: 13.0098
Epoch 7/40
59/59 [==============================] - 11s 189ms/step - loss: 11.5522
Epoch 8/40
59/59 [==============================] - 12s 195ms/step - loss: 10.7009
Epoch 9/40
59/59 [==============================] - 13s 220ms/step - loss: 10.2071
Epoch 10/40
59/59 [==============================] - 13s 220ms/step - loss: 9.1135
Epoch 11/40
59/59 [==============================] - 13s 222ms/step - loss: 8.6043
Epoch 12/40
59/59 [==============================] - 13s 223ms/step - loss: 7.9918
Epoc

In [19]:
result = model_resnet50_full.predict(test_image)


In [ ]:
x = []
y = []
for i in result:
   x.append(i[0])
   y.append(i[1]) 

In [ ]:
test_result = pd.read_csv("data/imagenames.csv")

In [ ]:
test_result["x"] = x
test_result["y"] = y

In [ ]:
test_result.to_csv("result/CNN_baseline_resnet50.csv",index=False)